In [1]:
import os 
import argparse
import GEOparse
import pandas as pd
import numpy as np
import pysradb
import subprocess as sp

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

gse_id = 'GSE101498'
gse_id = 'GSE178598'
gse_id = 'GSE151193'
gse_id = 'GSE133227'
gse_id = 'GSE147854'
gse_id = 'GSE173699'
gse_id = 'GSE147646'
gse_id = 'GSE136090'
gse_id = 'GSE154512'
gse_id = 'GSE190690' # not pubished yet
gse_id = 'GSE179755'
gse_id = 'GSE166232'
gse_id = 'GSE116193'

/mnt/BioHome/jreyna/software/anaconda3/hichip-db/lib/python3.10/site-packages/pysradb/utils.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Setting the output path

In [2]:
# # make an output directory
outdir = 'results/hichip_db/gse/'
os.makedirs(outdir, exist_ok=True)

# setting the output filename
output = os.path.join(outdir, "GSE_Query.{}.tsv".format(gse_id))

In [3]:
print('Writing the output to: "{}".'.format(output))

Writing the output to: "results/hichip_db/gse/GSE_Query.GSE116193.tsv".


## Query GEO for GSE metadata

In [4]:
# query the current GSE ID
geo_query = GEOparse.get_GEO(geo=gse_id, destdir=outdir, include_data=True, silent=True)

In [5]:
# parse through the information and make a useful table
gsm_data = []
for gsm_id, gsm in geo_query.gsms.items():
    
    title = '; '.join(gsm.metadata['title'])
    organism = ', '.join(gsm.metadata['organism_ch1'])
    source = ', '.join(gsm.metadata['source_name_ch1'])
    
    if 'description' in gsm.metadata:
        description = '; '.join(gsm.metadata['description'])
    else:
        description = ''

    
    for sra_link in gsm.relations['SRA']:
        # extracting the title, organism, source and description
        info = [gse_id,
                gsm_id,
                title,
                organism,
                source,
                description,
                sra_link]
        gsm_data.append(info)

gsm_data = pd.DataFrame(gsm_data)
gsm_data.columns = ['geo_id', 'gsm_id', 'title', 'organism', 'source', 'description', 'srx_link']

# extract SRA ID's
sra_ids = gsm_data['srx_link'].str.extract('(SRX[0-9]+)').squeeze()
gsm_data['srx_id'] = sra_ids

In [6]:
# loading the SRA tool
sra_querytool = pysradb.sraweb.SRAweb()

# query the SRA 
sra_query = sra_querytool.sra_metadata(gsm_data['srx_id'].values.tolist(), expand_sample_attributes=True)

In [7]:
meta = pd.merge(gsm_data,
                sra_query, left_on='srx_id',
                right_on='experiment_accession',
                suffixes=['_geo', '_sra'])

# calculating the number reads using the total number of spots
meta.loc[:, 'num_reads'] = meta.loc[:, 'total_spots'].astype(int) * 2

## Save the merged dataframe with all fields

In [8]:
meta_fn = os.path.join(outdir, '{}.meta.all_columns.xlsx'.format(gse_id))
meta.to_excel(meta_fn, index=False)

## Save the merged dataframe with most important columns

In [9]:
# most of these dropped columns are not needed, empty and some are redundant (specified)
# these columns are not dropped explicity with a drop call, but rather I extract only 
# the final columns I am interesting. The list below serves a book keeping purpose. 
drop_cols = ['sample_title', # empty
             'sample_organism', # redundant with organism
             'organism_taxid', 
             'library_name',
             'instrument',
             'instrument_model',
             'instrument_model_desc',
             'srx_link',
             'srx_id',
             'sample_accession',
             'study_accession',
             'study_title',
             'experiment_accession',
             'experiment_title',
             'experiment_desc',
             'organism_taxid',
             'library_name',
             'library_strategy',
             'library_source',
             'library_selection',
             'instrument',
             'instrument_model',
             'instrument_model_desc',
             'total_size',
             'run_total_spots',
             'run_total_bases', 
             'total_spots',
             'library_layout'] # not needed since all HiC data has to be completed with paired data

In [10]:
# setting the final columns
final_cols = ['geo_id',
             'gsm_id',
             'run_accession',
             'title',
             'source',
             'description',
             'organism',
             'num_reads']

# saving a table with the original column names
orig_cols_fn = os.path.join(outdir, '{}.meta.major_columns.original.xlsx'.format(gse_id))
meta[final_cols].to_excel(orig_cols_fn, index=False)

In [11]:
# extracting the final data
final_df = meta[final_cols]
final_df.columns = ['GSE ID', 'GSM ID', 'SRR ID',
                    'GEO Title', 'GEO Source', 'GEO Description',
                    'Organism', 'Number of Reads']

In [12]:
# saving a table with the new column names
renamed_cols_fn = os.path.join(outdir, '{}.meta.major_columns.renamed.xlsx'.format(gse_id))
final_df.to_excel(renamed_cols_fn, index=False)

In [13]:
final_df

,GSE ID,GSM ID,SRR ID,GEO Title,GEO Source,GEO Description,Organism,Number of Reads
0,GSE116193,GSM3212819,SRR7417408,lgs101293_H3K27ac ChIP-seq,H3K27ac ChIP-seq,H3K27ac ChIP-Seq experiment,Homo sapiens,160222938
1,GSE116193,GSM3212820,SRR7417409,lgs301283_H3K27ac ChIP-seq,H3K27ac ChIP-seq,H3K27ac ChIP-Seq experiment,Homo sapiens,186275286
2,GSE116193,GSM3212821,SRR7417410,lgs301430_H3K27ac ChIP-seq,H3K27ac ChIP-seq,H3K27ac ChIP-Seq experiment,Homo sapiens,194132394
3,GSE116193,GSM3212822,SRR7417411,lgs304050_H3K27ac ChIP-seq,H3K27ac ChIP-seq,H3K27ac ChIP-Seq experiment,Homo sapiens,140161088
4,GSE116193,GSM3212823,SRR7417412,lgs302634_H3K27ac ChIP-seq,H3K27ac ChIP-seq,H3K27ac ChIP-Seq experiment,Homo sapiens,173175154
...,...,...,...,...,...,...,...,...
107,GSE116193,GSM3212926,SRR7417515,lgs101645_CTCF HiChIP-seq,CTCF HiChIP-seq,CTCF-mediated HiChIP-Seq; processed data: CTCF...,Homo sapiens,101360268
108,GSE116193,GSM3212927,SRR7417516,lgs102580_CTCF HiChIP-seq,CTCF HiChIP-seq,CTCF-mediated HiChIP-Seq; processed data: CTCF...,Homo sapiens,214085924
109,GSE116193,GSM3212928,SRR7417517,lgs102943_CTCF HiChIP-seq,CTCF HiChIP-seq,CTCF-mediated HiChIP-Seq; processed data: CTCF...,Homo sapiens,70671496
110,GSE116193,GSM3212929,SRR7417518,lgs301315_CTCF HiChIP-seq,CTCF HiChIP-seq,CTCF-mediated HiChIP-Seq; processed data: CTCF...,Homo sapiens,78752034
